<a href="https://colab.research.google.com/github/matteosodano/MCLNN_AudioClassification/blob/master/HW02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Import { vertical-output: true }
%tensorflow_version 1.x

from google.colab import drive

import keras
from keras import applications, optimizers, regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Conv2D, AveragePooling2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Input, Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

import sklearn.metrics 
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
#@title Training and Test Set { vertical-output: true }
drive.mount('/content/drive')

datadir = '/content/drive/My Drive/Homework2'
trainingset = datadir+'/TrainingSet'
testset = datadir + '/TrainingSet6'
smarti_set = datadir + '/SMART-I'

batch_size = 64
input_shape = ()
train_datagen = ImageDataGenerator(
    # Generate batches of tensor image data with real-time data augmentation. 
    # The data will be looped over (in batches)
    rescale = 1. / 255,\
    # rescaling factor. Defaults to None. If None or 0, no rescaling is applied, 
    # otherwise we multiply the data by the value provided
    zoom_range=0.1,\
    # Float or [lower, upper]. Range for random zoom
    rotation_range=10,\
    # Degree range for random rotations
    width_shift_range=0.1,\
    # Fraction of total width
    height_shift_range=0.1,\
    # Fraction of total height
    horizontal_flip=False,\
    # Randomly flip inputs horizontally
    vertical_flip=True
    # Randomly flip inputs vertically
)

train_generator = train_datagen.flow_from_directory(
    # Takes the path to a directory & generates batches of augmented data
    directory=trainingset,
    # String, path to the target directory. It should contain one subdirectory per class 
    target_size=(150, 240),
    # Tuple of integers <height, width>
    color_mode="rgb",
    # The images will be converted to have 3 channels
    batch_size=batch_size,
    # Size of the batches of data
    class_mode="categorical",
    # "categorical" returns 2D array one-hot encoded labels
    shuffle=True
    # Shuffle the data
)

test_datagen = ImageDataGenerator(
    rescale = 1. / 255)

test_generator = test_datagen.flow_from_directory(
    directory=testset,
    target_size=(150, 240),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

smarti_datagen = ImageDataGenerator(
    rescale = 1. / 255)

smarti_generator = smarti_datagen.flow_from_directory(
    directory=smarti_set,
    target_size=(150, 240),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

num_samples = train_generator.n
num_classes = train_generator.num_classes
input_shape = train_generator.image_shape

classnames = [k for k,v in train_generator.class_indices.items()]

print("Image input %s" %str(input_shape))
print("Classes: %r" %classnames)

print('Loaded %d training samples from %d classes.' %(num_samples,num_classes))
print('Loaded %d test samples from %d classes.' %(test_generator.n,test_generator.num_classes))

In [0]:
#@title Random Images { vertical-output: true }
n = len(classnames)
x,y = train_generator.next()
# x,y size is train_generator.batch_size

for i in range(0,n):
    image = x[i]
    label = y[i].argmax() 
    print(classnames[label])
    plt.imshow(image)
    plt.show()


In [0]:
#@title Neural Network { vertical-output: true }
def MatNet(input_shape, num_classes):

    model = Sequential()

    # C1 Convolutional Layer --- 2D convolution layer for spatial convolution over images
    # 1. Dimensionality of the output space
    # 2. When using this layer as the first layer in a model, provide the keyword argument input_shape
    # 3. Size of the convolution kernel
    # 4. An integer or tuple/list of 2 integers, specifying the strides of the 
    #    convolution along the height and width. Stride = how much do I slide
    # 5. Padding = keep the same size by adding zeros
    # 6. Activation function
    model.add(Conv2D(32, input_shape=input_shape, kernel_size=(7,7), strides=(3,3), 
                     padding='same', activation='tanh'))
    
    # Invariance to local translations. Reduce the importance of detecting an information
    # in a precise pixel 
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    # Normalize the activations of the previous layer at each batch, i.e. applies a transformation 
    # that maintains the mean activation close to 0 and the activation standard deviation close to 1.
    model.add(BatchNormalization())


    # C2 Convolutional Layer 
    model.add(Conv2D(64, kernel_size=(6,6), strides=(2,2), padding='same', 
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    model.add(BatchNormalization())


    # C3 Convolutional Layer
    model.add(Conv2D(96, kernel_size=(2,2), strides=(1,1), padding='valid', 
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())


    # Flatten: transforms the data representation from a 3D space to a 1D vector
    model.add(Flatten())
    
    # D1 Dense Layer: regular densely-connected NN layer
    # 1. Number of units
    # 2. Activation function
    model.add(Dense(300, activation='relu'))

    # Dropout consists in randomly setting a fraction of input units to 0 
    # at each update during training time, which helps prevent overfitting.
    model.add(Dropout(0.33))

    model.add(BatchNormalization())


    # D2 Dense Layer
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.33))
    model.add(BatchNormalization())


    # D3 Dense Layer
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.25))
    model.add(BatchNormalization())


    # D4 Dense Layer
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.25))
    model.add(BatchNormalization())


    # Output Layer
    model.add(Dense(num_classes, activation='softmax'))


    # Compile
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), 
                  metrics=['accuracy'])

    return model
 
# create the model
model = MatNet(input_shape,num_classes)
model.summary()

In [0]:
#@title Network Model { vertical-output: true }
tf.keras.utils.plot_model(model)


In [0]:
#@title Training { vertical-output: true }

stopping = EarlyStopping(monitor='val_acc', patience=3)

steps_per_epoch=train_generator.n//train_generator.batch_size
val_steps=test_generator.n//test_generator.batch_size+1

try:
    # Trains the model on data generated batch-by-batch by a Python generator
    # 1. Generator that gives tuples, that form batches
    # 2. Integer number of epochs to train the model. An epoch is an iteration over the entire data provided
    # 3. number of steps (batches of samples) to yield from generator before declaring one epoch 
    #    finished and starting the next epoch
    # 4. Generator for the test set
    # 5. Total number of steps (batches of samples) to yield from test generator 
    #    before stopping at the end of every epoch
    history = model.fit_generator(train_generator, epochs=30, verbose=1, \
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=smarti_generator,\
                    validation_steps=val_steps)
except KeyboardInterrupt:
    pass

In [0]:
#@title Save Model { vertical-output: true }
models_dir = '/content/drive/My Drive/Homework2'

def savemodel(model,problem):
    filename = os.path.join(models_dir, '%s.h5' %problem)
    model.save(filename)
    print("\nModel saved successfully on file %s\n" %filename)

# Save the model
savemodel(model,'HW02_MatNet')

In [0]:
#@title Performance { vertical-output: true }
preds = model.predict_generator(test_generator,verbose=1,steps=val_steps)

Ypred = np.argmax(preds, axis=1)
Ytest = test_generator.classes 

print(classification_report(Ytest, Ypred, labels=None, target_names=classnames, digits=3))

In [0]:
#@title Confusion Matrix { vertical-output: true }
def plot_confusion_matrix(cm, classes):
    cm = pd.DataFrame(cm, index=classes, columns=classes)

    heatmap = sns.heatmap(cm, annot=True, fmt="d")
    bottom, top = heatmap.get_ylim()
    heatmap.set_ylim(bottom + 0.5, top - 0.5)
  
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


cnf_matrix = confusion_matrix(Ytest, Ypred)
classes = ['Haze', 'Rainy', 'Snowy', 'Sunny']

plot_confusion_matrix(cnf_matrix, classes)
plt.show()

In [0]:
#@title Accuracy and Loss plots { vertical-output: true }
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
#@title Transfer Network { vertical-output: true }

# Inception Net
transfer_model = tf.keras.applications.InceptionResNetV2(\
    input_shape=input_shape,include_top=False, weights='imagenet', pooling='avg')
model = tf.keras.Sequential([transfer_model, tf.keras.layers.Flatten(), 
                             tf.keras.layers.Dense(num_classes, activation='softmax')])

# VGG16 Net
#transfer_model = tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet',
#                                                   input_shape=input_shape,pooling='max')
#model = tf.keras.Sequential([transfer_model,tf.keras.layers.Flatten(),
#                             tf.keras.layers.Dense(num_classes, activation='softmax')])

optimizer = 'adam'
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])

model.summary()

In [0]:
#@title Train Transfer Net { vertical-output: true }
# fit the transferNet on the training data
from keras.callbacks import ModelCheckpoint, EarlyStopping

#stopping = EarlyStopping(monitor='val_acc', patience=3)

steps_per_epoch = train_generator.n//train_generator.batch_size
val_steps = test_generator.n//test_generator.batch_size+1

try:
    history_transfer = model.fit_generator(train_generator, epochs=10, verbose=1,
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=test_generator,\
                    validation_steps=val_steps)
except KeyboardInterrupt:
    pass

In [0]:
#@title Performance { vertical-output: true }
preds_t = model.predict_generator(test_generator,verbose=1,steps=val_steps)

Ypred = np.argmax(preds_t, axis=1)
Ytest = test_generator.classes 

print(classification_report(Ytest, Ypred, labels=None, target_names=classnames, digits=3))

In [0]:
#@title Confusion Matrix { vertical-output: true }
def plot_confusion_matrix(cm, classes):
    cm = pd.DataFrame(cm, index=classes, columns=classes)

    heatmap = sns.heatmap(cm, annot=True, fmt="d")
    bottom, top = heatmap.get_ylim()
    heatmap.set_ylim(bottom + 0.5, top - 0.5)
  
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


cnf_matrix = confusion_matrix(Ytest, Ypred)
classes = ['Haze', 'Rainy', 'Snowy', 'Sunny']

plot_confusion_matrix(cnf_matrix, classes)
plt.show()

In [0]:
#@title Accuracy and Loss plots { vertical-output: true }

plt.plot(history_transfer.history['acc'])
plt.plot(history_transfer.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_transfer.history['loss'])
plt.plot(history_transfer.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
#@title Save Model { vertical-output: true }
models_dir = '/content/drive/My Drive/Homework2'

def savemodel(model,problem):
    filename = os.path.join(models_dir, '%s.h5' %problem)
    model.save(filename)
    print("\nModel saved successfully on file %s\n" %filename)

# Save the model
savemodel(model,'InceptionNet')

In [0]:
#@title Performances { vertical-output: true }
#model.predict(smarti_generator)

preds_t = model.predict_generator(smarti_generator)

Ypred = np.argmax(preds_t, axis=1)
Ytest = smarti_generator.classes 
#print(preds_t.shape, Ypred.shape, Ytest.shape)
print(classification_report(Ytest, Ypred, labels=None, target_names=classnames, digits=3))

In [0]:
#@title Confusion Matrix { vertical-output: true }
cnf_matrix = confusion_matrix(Ytest, Ypred)
classes = ['Haze', 'Rainy', 'Snowy', 'Sunny']

plot_confusion_matrix(cnf_matrix, classes)
plt.show()

In [0]:
#@title Blind Test Set { vertical-output: true }
blind_set = datadir + '/WeatherBlindTestSet'

blind_datagen = ImageDataGenerator(
    rescale = 1. / 255)

blind_generator = blind_datagen.flow_from_directory(
    directory=blind_set,
    target_size=(150, 240),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

# load the best model
model = load_model(datadir+'/InceptionNet.h5')

blind_pred = model.predict(blind_generator)
predictions = np.argmax(blind_pred, axis=1)

In [0]:
#@title Write CSV { vertical-output: true }
def write_csv(pred):
  drive.mount('/content/drive')
  with open('/content/drive/My Drive/Homework2/1695082.csv', 'w') as csv:
    len_p = len(predictions)
    for i in range(0,len_p):
      data = predictions[i]
      if data == 0:
        csv.write('HAZE\n')
      elif data == 1:
        csv.write('RAINY\n')
      elif data == 2:
        csv.write('SNOWY\n')
      elif data == 3:
        csv.write('SUNNY\n')
  drive.flush_and_unmount()
  return 0

write_csv(predictions)